In [1]:
%matplotlib inline

In [33]:
# Import libs

import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.fftpack import fft,ifft
from numpy.fft import rfft,irfft
from scipy.cluster.vq import vq,kmeans,whiten
from scipy import conj,argmax

In [52]:
# This is to set log file path

logFileName ='000-beetle-1.53-4.1'
logFileType = '.BBL'
logFilePath = logFileName+logFileType

In [70]:
# This cell uses OrangeBox to decode blackbox data

from orangebox import Parser

# Load a file
parser = Parser.load(logFilePath)
# or optionally select a log by index (1 is the default)
# parser = Parser.load("btfl_all.bbl", 1)

# Print headers
#print("headers:", parser.headers)

# Print the names of fields
print("field names:", parser.field_names)

# Select a specific log within the file by index
print("log count:", parser.reader.log_count)
#parser.set_log_index(1)

# Complete list of events only available once all frames have been parsed
#print("events:", parser.events)

# Selecting another log changes the header and frame data produced by the Parser
# and also clears any previous results and state
parser.set_log_index(1)

field names: ['loopIteration', 'time', 'axisP[0]', 'axisP[1]', 'axisP[2]', 'axisI[0]', 'axisI[1]', 'axisI[2]', 'axisD[0]', 'axisD[1]', 'axisF[0]', 'axisF[1]', 'axisF[2]', 'rcCommand[0]', 'rcCommand[1]', 'rcCommand[2]', 'rcCommand[3]', 'setpoint[0]', 'setpoint[1]', 'setpoint[2]', 'setpoint[3]', 'vbatLatest', 'gyroADC[0]', 'gyroADC[1]', 'gyroADC[2]', 'accSmooth[0]', 'accSmooth[1]', 'accSmooth[2]', 'debug[0]', 'debug[1]', 'debug[2]', 'debug[3]', 'motor[0]', 'motor[1]', 'motor[2]', 'motor[3]', 'flightModeFlags', 'stateFlags', 'failsafePhase', 'rxSignalReceived', 'rxFlightChannelsValid']
log count: 1


In [71]:
#set a load_log_data funciton to help load log data from the logfile
def load_log_data(parser, *args):
    result_index = []
    result_data = []
    i=0
    for arraydef in args:
        if type(arraydef).__name__ == "str":
            result_index.append(parser.field_names.index(arraydef))
            result_data.append([])
        else:
            for name in arraydef:
                result_index.append(parser.field_names.index(name))
                result_data.append([])
    for frame in parser.frames():
        for i in range(len(result_index)):
            result_data[i].append(frame.data[result_index[i]])
    i = 0
    result = []
    for arraydef in args:
        if type(arraydef).__name__ == "str":
            result.append(np.array(result_data[i]))
            i=i+1
        else:
            part_array = []
            for name in arraydef:
                part_array.append(result_data[i])
                i=i+1
            result.append(np.array(part_array))
    return result

In [72]:
# Load data for analyze

loopIteration   = load_log_data(parser, 'loopIteration')
time            = load_log_data(parser, 'time')
setpointData    = load_log_data(parser, ('setpoint[0]',  'setpoint[1]', 'setpoint[2]'))
gyroADCData     = load_log_data(parser, ('gyroADC[0]',   'gyroADC[1]',  'gyroADC[2]'))
gyroScaleData   = load_log_data(parser, ('debug[0]', 'debug[1]', 'debug[2]'))
thrData         = load_log_data(parser, 'rcCommand[3]')
#[loopIteration, time, setpointData, gyroADCData, gyroScaleData, thrData] = load_log_data(parser, 'loopIteration', 'time', ('setpoint[0]',  'setpoint[1]', 'setpoint[2]'),('gyroADC[0]',   'gyroADC[1]',  'gyroADC[2]'), ('debug[0]', 'debug[1]', 'debug[2]'), 'rcCommand[3]')

Unknown event type: 110
Unknown event type: 110


In [73]:
print(time)
time=time-time[0]
print(time)
diffOfTime = np.diff(time)
print(diffOfTime)
averageUs = np.median(diffOfTime)
print(averageUs)
samplingRate = 1000000/averageUs
print(samplingRate)

[array([], dtype=float64)]
[]
[]
nan
nan
